In [1]:
from function import *

#### parse data

In [2]:
start_time_str = '2022-11-21T00:00:00Z'
end_time_str = '2024-12-06T00:00:00Z'
symbol = 'BTC/USDT'
bull_btc_1h_df = batch_load_hr_data(start_time_str, end_time_str, symbol)

In [3]:
bull_btc_1h_df.tail()

,timestamp,open,high,low,close,volume,datetime
17936,1733562000000,99414.76,99821.83,99414.75,99676.48,558.38011,2024-12-07 09:00:00
17937,1733565600000,99676.48,99708.43,99440.00,99529.50,795.39219,2024-12-07 10:00:00
17938,1733569200000,99529.50,99750.00,99421.53,99620.00,437.99027,2024-12-07 11:00:00
17939,1733572800000,99620.00,99703.55,99364.00,99426.87,421.89864,2024-12-07 12:00:00
17940,1733576400000,99426.88,99518.75,99304.00,99328.39,333.81641,2024-12-07 13:00:00


#### check intervals

In [7]:
#1 interval is skipped, need to input
bull_btc_1h_df['timestamp'].diff().value_counts()

timestamp
3600000.0    17939
7200000.0        1
Name: count, dtype: int64

In [8]:
#find its index
bull_btc_1h_df[bull_btc_1h_df['timestamp'].diff()==7_200_000]

,timestamp,open,high,low,close,volume,datetime
2965,1679666400000,28079.99,28253.01,27835.0,27989.06,8983.24018,2023-03-24 14:00:00


In [11]:
#validate the skip
bull_btc_1h_df.loc[2964:2967,:]

,timestamp,open,high,low,close,volume,datetime
2964,1679659200000,28080.00,28080.00,28080.00,28080.00,0.00000,2023-03-24 12:00:00
2965,1679666400000,28079.99,28253.01,27835.00,27989.06,8983.24018,2023-03-24 14:00:00
2966,1679670000000,27989.07,28076.82,27843.41,28018.04,5198.28681,2023-03-24 15:00:00
2967,1679673600000,28018.04,28059.63,27831.33,27831.33,3377.68679,2023-03-24 16:00:00


In [12]:
#seems something odd happened with the exchange at 2023-03-24 12:00 
# as no trades were recorded we can drop this row
bull_btc_1h_clean_df = bull_btc_1h_df.drop([2964], axis=0).reset_index(drop=True)

#### check nans and zeros

In [13]:
#no nans
bull_btc_1h_clean_df.isna().sum()

timestamp    0
open         0
high         0
low          0
close        0
volume       0
datetime     0
dtype: int64

In [19]:
#min values greater than 0
bull_btc_1h_clean_df.describe()

,timestamp,open,high,low,close,volume,datetime
count,1.794000e+04,17940.000000,17940.000000,17940.000000,17940.000000,17940.000000,17940
mean,1.701285e+12,44028.599110,44180.567976,43871.367088,44033.228769,3176.385866,2023-11-29 19:10:10.434782720
min,1.668989e+12,15648.230000,15769.990000,15476.000000,15649.520000,92.165510,2022-11-21 00:00:00
25%,1.685141e+12,26950.465000,27043.447500,26881.365000,26954.102500,735.550885,2023-05-26 22:45:00
50%,1.701286e+12,37861.545000,37966.225000,37768.340000,37863.000000,1305.319305,2023-11-29 19:30:00
75%,1.717431e+12,62946.507500,63185.220000,62739.600000,62948.930000,3105.898080,2024-06-03 16:15:00
max,1.733576e+12,103508.000000,104088.000000,102684.730000,103507.990000,80041.752040,2024-12-07 13:00:00
std,1.864603e+10,20317.546425,20405.043897,20225.630047,20320.681285,5127.484584,NaN


#### save results

In [20]:
bull_btc_1h_clean_df.to_csv('clean_data/btc_1h.csv', index=False)